In [1]:
import uuid
import os
import sys
from typing import Optional, List

from pydantic import Field
from dotenv import load_dotenv
from rich.pretty import pprint
from datetime import datetime
from zoneinfo import ZoneInfo

from zep_cloud.client import AsyncZep
from zep_cloud.types import Message
from zep_cloud.extractor import (
    ZepModel,
    ZepText,
    ZepNumber,
    ZepFloat,
    ZepRegex,
    ZepZipCode,
    ZepDateTime,
    ZepEmail,
    ZepPhoneNumber,
)

In [2]:
sys.path.append(os.path.join(os.getcwd(), "..", "..", "data"))

from chat_history_shoe_purchase import history

In [3]:
# Place your Zep API key in your environemnt or in a .env file
load_dotenv()

API_KEY = os.getenv("ZEP_API_KEY")

assert API_KEY is not None, "ZEP_API_KEY is not set"

In [4]:
client = AsyncZep(
    api_key=API_KEY,
)
user_id = uuid.uuid4().hex  # unique user id. can be any alphanum string

await client.user.add(
    user_id=user_id,
    email="user@example.com",
    first_name="Jane",
    last_name="Smith",
    metadata={"vip": "true"},
)

session_id = uuid.uuid4().hex  # unique session id. can be any alphanum string

# Create session associated with the above user
await client.memory.add_session(
    session_id=session_id, user_id=user_id, metadata={"foo": "bar"}
)

# Add Memory for session
for m in history[:-1]:
    await client.memory.add(session_id=session_id, messages=[Message(**m)])

In [5]:
# Check that the messages were added
msgs = await client.memory.get_session_messages(session_id)

pprint([(msg.role, msg.content) for msg in msgs.messages])

[
│   ('user', "Hi there, I'm looking for a new pair of shoes. I'm not sure where to start."),
│   (
│   │   'assistant',
│   │   "Hello! I'd be happy to help you find the perfect pair of shoes. To get started, could you tell me what brand you're interested in, or if you're open to any brand?"
│   ),
│   ('user', "I like Nike and Adidas, but I'm open to other brands too."),
│   (
│   │   'assistant',
│   │   "That's great! It's important to find the right shoe for your needs. Could you tell me what you'll be using these shoes for mainly? For example, are they for running, basketball, or casual wear?"
│   ),
│   ('user', "I'll be using them for running."),
│   (
│   │   'assistant',
│   │   'Fantastic! Now, could you please provide me with your shoe size and foot width? This information will help me narrow down the options for you.'
│   ),
│   ('user', "I wear a men's size 10, and I'm not sure about the foot width."),
│   (
│   │   'assistant',
│   │   "Thank you. Based on your size, I'll make sure to suggest shoes with a wide range of sizes and widths available."
│   ),
│   ('user', 'I have a bit of a pronation issue. Do I need to consider that while choosing shoes?'),
│   (
│   │   'assistant',
│   │   "Yes, pronation is essential when choosing running shoes. It refers to the way your foot rolls inward or outward when you walk or run. For pronation issues, stability or motion control shoes are usually recommended. I'll make sure to consider this in my suggestions."
│   ),
│   ('user', 'Great, I have a budget of around $120.'),
│   (
│   │   'assistant',
│   │   "Perfect! I'll make sure to find the best running shoes that fit your preferences, taking into account your brand preferences, running needs, size, and budget."
│   ),
│   (
│   │   'assistant',
│   │   "I've found a few great options for you. The first one is the Brooks Ghost 14, which is known for its cushioning and support, ideal for runners with pronation issues. It's available in your size, and it's within your budget."
│   ),
│   ('user', "I've heard good things about Brooks, but I was kind of hoping to get a Nike or an Adidas shoe."),
│   (
│   │   'assistant',
│   │   "I understand. In that case, I'd recommend the Adidas Ultraboost 21. It's a popular choice among runners for its comfort, energy return, and stability. Plus, it's on sale at $129.99 today, almost within your budget."
│   ),
│   ('user', "They're nice looking. Yes, I think I'll go with those.")
]

In [6]:
LAST_N = 10 # The number of most recent messages to consider for extraction

class SalesOrder(ZepModel):
    order_id: Optional[ZepNumber] = Field(description="The order id", default=None)
    order_items: Optional[ZepText] = Field(description="The item ordered", default=None)
    delivery_date: Optional[ZepDateTime] = Field(
        description="The date and time the order will be delivered", default=None
    )
    order_amount: Optional[ZepFloat] = Field(
        description="The ordered item's price", default=None
    )
    brand_preferences: Optional[ZepRegex] = Field(
        description="The customer's preferred brands. Comma-separated list of brands",
        default=None,
        pattern=r"\w+(, \w+)+",
    )
    shoe_size: Optional[ZepNumber] = Field(
        description="The customer's shoe size", default=None
    )
    customer_first_name: Optional[ZepText] = Field(
        description="The customer's first name. Capitalize appropriately.", default=None
    )
    customer_last_name: Optional[ZepText] = Field(
        description="The customer's last name. Capitalize appropriately.", default=None
    )
    customer_email: Optional[ZepEmail] = Field(
        description="The customer's email", default=None
    )
    customer_phone: Optional[ZepPhoneNumber] = Field(
        description="The customer's phone number", default=None
    )
    street_number: Optional[ZepNumber] = Field(
        description="The delivery street number", default=None
    )
    street_name: Optional[ZepText] = Field(
        description="The delivery street name", default=None
    )
    zip_code: Optional[ZepZipCode] = Field(
        description="The delivery zip code", default=None
    )
    order_currency: Optional[ZepRegex] = Field(
        description="The order currency: USD, GBP, or UNKNOWN",
        default=None,
        pattern=r"(UNKNOWN|USD|GBP)",
    )

In [7]:
def merge_models(
    existing_order: SalesOrder,
    new_order: SalesOrder,
    mutable_fields: Optional[List[str]] = None,
) -> SalesOrder:
    """Merge two ZepModels, overwriting existing fields with new values. We exclude unset, none, and defaults.
    All fields in existing_order are immutable by default, unless specified in mutable_fields.
    """
    if mutable_fields is None:
        mutable_fields = []

    existing_data = existing_order.model_dump(
        exclude_unset=True, exclude_none=True, exclude_defaults=True
    )
    new_data = new_order.model_dump(
        exclude_unset=True, exclude_none=True, exclude_defaults=True
    )

    for field in list(existing_data.keys()):
        if field not in mutable_fields:
            new_data.pop(field, None)

    return SalesOrder(
        **{
            **existing_data,
            **new_data,
        }
    )

In [8]:
# Extract the sales order information from the session memory
sales_order = await client.memory.extract(
    session_id,  # The unique identifier for the user's session
    SalesOrder,  # The model class to use for extracting the data
    last_n=LAST_N,  # The number of most recent messages to consider for extraction
    current_date_time=datetime.now(
        ZoneInfo("America/New_York")
    ),  # The current date and time in the specified timezone, will be used to resolve relative dates.
)
pprint(sales_order)

SalesOrder(
│   order_id=14,
│   order_items='Adidas Ultraboost 21',
│   delivery_date=None,
│   order_amount=129.99,
│   brand_preferences='Nike, Adidas',
│   shoe_size=10,
│   customer_first_name=None,
│   customer_last_name=None,
│   customer_email=None,
│   customer_phone=None,
│   street_number=None,
│   street_name=None,
│   zip_code=None,
│   order_currency='USD'
)

In [9]:
# Extract AND validate the sales order information. Doing so adds a validation step evaluating whether the extracted data matches provided dialog context.
# The validation step adds additional latency and may result in false negatives i.e. data present in the dialog context may not be extracted.
sales_order_validated = await client.memory.extract(
    session_id,  # The unique identifier for the user's session
    SalesOrder,  # The model class to use for extracting the data
    last_n=LAST_N,  # The number of most recent messages to consider for extraction
    current_date_time=datetime.now(
        ZoneInfo("America/New_York")
    ),  # The current date and time in the specified timezone, will be used to resolve relative dates.
    validate=True,
)
pprint(sales_order_validated)

SalesOrder(
│   order_id=None,
│   order_items='Adidas Ultraboost 21',
│   delivery_date=None,
│   order_amount=129.99,
│   brand_preferences='Nike, Adidas',
│   shoe_size=10,
│   customer_first_name=None,
│   customer_last_name=None,
│   customer_email=None,
│   customer_phone=None,
│   street_number=None,
│   street_name=None,
│   zip_code=None,
│   order_currency='USD'
)

In [10]:
# Add an additional message with the user's email address. We'll also infer the first name.
await client.memory.add(
    session_id=session_id,
    messages=[
        Message(role_type="user", content="My email address is mary@example.com")
    ],
)

sales_order_new = await client.memory.extract(
    session_id,
    SalesOrder,
    last_n=LAST_N,
    current_date_time=datetime.now(ZoneInfo("America/New_York")),
)

# Since we're using a rolling message window to extract facts, we need to merge the new facts with the existing ones.
sales_order = merge_models(sales_order, sales_order_new)

pprint(sales_order)

SalesOrder(
│   order_id=14,
│   order_items='Adidas Ultraboost 21',
│   delivery_date=None,
│   order_amount=129.99,
│   brand_preferences='Nike, Adidas',
│   shoe_size=10,
│   customer_first_name='Mary',
│   customer_last_name=None,
│   customer_email='mary@example.com',
│   customer_phone=None,
│   street_number=None,
│   street_name=None,
│   zip_code=None,
│   order_currency='USD'
)

In [11]:
# Add an additional message with the user's street address and zip code.
await client.memory.add(
    session_id=session_id,
    messages=[
        Message(
            role_type="assistant",
            content="Great, what is your street address and zip code?",
        ),
        Message(role_type="user", content="987 Main St, zip code is 12345"),
    ],
)

sales_order_new = await client.memory.extract(
    session_id,
    SalesOrder,
    last_n=LAST_N,
    current_date_time=datetime.now(ZoneInfo("America/New_York")),
)

sales_order = merge_models(sales_order, sales_order_new)

pprint(sales_order)

SalesOrder(
│   order_id=14,
│   order_items='Adidas Ultraboost 21',
│   delivery_date=None,
│   order_amount=129.99,
│   brand_preferences='Nike, Adidas',
│   shoe_size=10,
│   customer_first_name='Mary',
│   customer_last_name=None,
│   customer_email='mary@example.com',
│   customer_phone=None,
│   street_number=987,
│   street_name='Main St',
│   zip_code='12345',
│   order_currency='USD'
)

In [12]:
# Add an additional message with a relative date. The relative date will be converted to an absolute date.
await client.memory.add(
    session_id=session_id,
    messages=[
        Message(
            role_type="assistant",
            content="Fantastic! We'll deliver your order tomorrow at 1pm!",
        ),
    ],
)

sales_order_new = await client.memory.extract(
    session_id,
    SalesOrder,
    last_n=LAST_N,
    current_date_time=datetime.now(ZoneInfo("America/New_York")),
)

sales_order = merge_models(sales_order, sales_order_new)

pprint(sales_order)

SalesOrder(
│   order_id=14,
│   order_items='Adidas Ultraboost 21',
│   delivery_date=datetime.datetime(2024, 7, 9, 13, 0),
│   order_amount=129.99,
│   brand_preferences='Nike, Adidas',
│   shoe_size=10,
│   customer_first_name='Mary',
│   customer_last_name=None,
│   customer_email='mary@example.com',
│   customer_phone=None,
│   street_number=987,
│   street_name='Main St',
│   zip_code='12345',
│   order_currency='USD'
)